In [1]:
%pip install kagglehub
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install tensorflow
%pip install matplotlib
%pip install tf-keras
%pip install imbalanced-learn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import kagglehub
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
dataset = pd.read_csv("/content/mtsamples.csv")
print("Head: ", dataset.head)

Head:  <bound method NDFrame.head of       Unnamed: 0                                        description  \
0              0   A 23-year-old white female presents with comp...   
1              1           Consult for laparoscopic gastric bypass.   
2              2           Consult for laparoscopic gastric bypass.   
3              3                             2-D M-Mode. Doppler.     
4              4                                 2-D Echocardiogram   
...          ...                                                ...   
4994        4994   Patient having severe sinusitis about two to ...   
4995        4995   This is a 14-month-old baby boy Caucasian who...   
4996        4996   A female for a complete physical and follow u...   
4997        4997   Mother states he has been wheezing and coughing.   
4998        4998   Acute allergic reaction, etiology uncertain, ...   

                medical_specialty                                sample_name  \
0            Allergy / Immunol

## Data processing

- We drop every other column except transcription and medical_specialty.
- We also drop any rows with empty or null transcription or medical_specialty.

- Then we drop all the classes in the excluded specialties list below. We do this as these are general terms and don't specifically map to any specialty.
- We then merge the classes with large overlaps - e.g. Neurosurgery and neurology, Neurosurgery is a subset of neurology.

In [4]:

dataset.dropna(subset=['transcription', 'medical_specialty'], inplace=True)

dataset = dataset[['transcription', 'medical_specialty']]

specialty_counts = dataset['medical_specialty'].value_counts()
valid_specialties = specialty_counts[specialty_counts >= 30].index
dataset = dataset[dataset['medical_specialty'].isin(valid_specialties)]

dataset['medical_specialty'] = dataset['medical_specialty'].str.strip()

excluded_specialties = [
    'Surgery',
    'SOAP / Chart / Progress Notes',
    'Office Notes',
    'Consult - History and Phy.',
    'Emergency Room Reports',
    'Discharge Summary',
    'Pain Management',
    'General Medicine',
    'Radiology',
]

dataset = dataset[~dataset['medical_specialty'].isin(excluded_specialties)]

category_mapping = {
    'Neurosurgery': 'Neurology',
    'Nephrology': 'Urology',
}

dataset['medical_specialty'] = dataset['medical_specialty'].replace(category_mapping)

for i, (category_name, category) in enumerate(dataset.groupby("medical_specialty")):
    print(f"Category {i}: {category_name}: {len(category)}")


Category 0: Cardiovascular / Pulmonary: 371
Category 1: ENT - Otolaryngology: 96
Category 2: Gastroenterology: 224
Category 3: Hematology - Oncology: 90
Category 4: Neurology: 317
Category 5: Obstetrics / Gynecology: 155
Category 6: Ophthalmology: 83
Category 7: Orthopedic: 355
Category 8: Pediatrics - Neonatal: 70
Category 9: Podiatry: 47
Category 10: Psychiatry / Psychology: 53
Category 11: Urology: 237


## Data processing

- We clean the text and then tokenize it and then lemmatize all the words in it.

In [5]:
from sklearn.model_selection import train_test_split

def clean_text(text):
    lemmatizer = WordNetLemmatizer()
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(text) if word not in stopwords.words('english')])
    return text

dataset['processed_transcription'] = dataset['transcription'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(
    dataset['processed_transcription'], dataset['medical_specialty'], test_size=0.2, random_state=42, stratify=dataset['medical_specialty']
)

LSTM

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalMaxPooling1D


embedding_dim = 128
lstm_units = 64
max_len = 256
max_words = 50000
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(set(y_train)), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_padded, to_categorical(y_train_encoded), epochs=40, batch_size=32, validation_split=0.2)

y_pred_probs = model.predict(X_test_padded)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = y_test_encoded

print("Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.1656 - loss: 2.4166 - val_accuracy: 0.1994 - val_loss: 2.2811
Epoch 2/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.2088 - loss: 2.1339 - val_accuracy: 0.2679 - val_loss: 2.0179
Epoch 3/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.3715 - loss: 1.7173 - val_accuracy: 0.4435 - val_loss: 1.7164
Epoch 4/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5790 - loss: 1.3090 - val_accuracy: 0.4167 - val_loss: 1.7428
Epoch 5/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.7158 - loss: 0.8990 - val_accuracy: 0.4702 - val_loss: 1.7377
Epoch 6/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7330 - loss: 0.8321 - val_accuracy: 0.4911 - val_loss: 1.8271
Epoch 7/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7997 - loss: 0.6549 - val_accuracy: 0.4970 - val_loss: 1.8357
Epoch 8/40
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.8051 - loss: 0.5705 - val_accuracy: 0.4970 - val_loss: 1.

BERT embedding

In [8]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(texts):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
        with torch.no_grad():
            outputs = bert(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(cls_embedding)
    return np.array(embeddings)

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y_train)

X_train_bert = get_bert_embeddings(X_train)
X_test_bert = get_bert_embeddings(X_test)

y_train_cat = to_categorical(y_encoded)
y_test_encoded_cat = label_encoder.transform(y_test)
y_test_cat = to_categorical(y_test_encoded_cat)

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(768,)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_bert, y_train_cat, epochs=20, batch_size=32, validation_split=0.2)

y_pred_probs = model.predict(X_test_bert)
y_pred = np.argmax(y_pred_probs, axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test_encoded_cat, y_pred, digits=4))

100%|██████████| 420/420 [03:09<00:00,  2.22it/s]
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 81ms/step - accuracy: 0.1722 - loss: 2.3542 - val_accuracy: 0.2232 - val_loss: 2.1914
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2916 - loss: 2.1482 - val_accuracy: 0.3036 - val_loss: 2.0525
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3029 - loss: 2.0089 - val_accuracy: 0.3363 - val_loss: 1.8502
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3974 - loss: 1.7995 - val_accuracy: 0.4464 - val_loss: 1.6781
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4284 - loss: 1.6910 - val_accuracy: 0.4613 - val_loss: 1.6167
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4965 - loss: 1.5148 - val_accuracy: 0.4583 - val_loss: 1.5593
Epoch 7/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4902 - loss: 1.5167 - val_accuracy: 0.5119 - val_loss: 1.4692
Epoch 8/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5199 - loss: 1.3750 - val_accuracy: 0.5476 - val_loss